In [1]:
"""
Population Vulnerability Analysis
Cell 1: Setup and check existing data
"""

import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from pathlib import Path
import sys

# Define paths directly
BASE_DIR = Path.cwd().parent
DATA_DIR = BASE_DIR / "data"
POPULATION_DIR = DATA_DIR / "raw" / "population"
PROCESSED_DATA_DIR = DATA_DIR / "processed"

print("=" * 60)
print("POPULATION DATA EXPLORATION")
print("=" * 60)

# Check what files we have
print("\n📁 Files in population directory:")
print(f"Path: {POPULATION_DIR}")
if POPULATION_DIR.exists():
    for file in sorted(POPULATION_DIR.iterdir()):
        if file.is_file():
            size_mb = file.stat().st_size / 1_000_000
            print(f"  - {file.name} ({size_mb:.2f} MB)")
else:
    print("  Directory not found!")

print("\n💡 We need population data by commune with age breakdowns.")
print("   Specifically: total population and population aged 65+")

POPULATION DATA EXPLORATION

📁 Files in population directory:
Path: /Users/antoineverhulst/Documents/Project/claude/heat_risk_france/data/raw/population
  - Filosofi2019_carreaux_200m_csv.7z (61.75 MB)
  - TD_POP1A_2020.zip (6.17 MB)
  - base-pop-legales-2023.zip (61.77 MB)
  - donnees_arrondissements.csv (0.02 MB)
  - donnees_cantons.csv (0.12 MB)
  - donnees_collectivites.csv (0.00 MB)
  - donnees_communes.csv (2.17 MB)
  - donnees_communes_deleguees.csv (0.18 MB)
  - donnees_departements.csv (0.01 MB)
  - donnees_fractions_cantonales.csv (0.05 MB)
  - donnees_regions.csv (0.00 MB)
  - metadonnees.csv (1.55 MB)
  - pop_age_ensemble.zip (1.13 MB)
  - population_age_10tranches_2020.xlsx (10.59 MB)

💡 We need population data by commune with age breakdowns.
   Specifically: total population and population aged 65+


In [2]:
"""
Cell 5: Download population by age groups from INSEE
This has the data we need: population by commune with age breakdowns!
"""

import requests

print("=" * 60)
print("DOWNLOADING POPULATION BY AGE DATA")
print("=" * 60)

# Direct link to complete population data with age breakdowns
url = "https://www.insee.fr/fr/statistiques/fichier/7739582/ensemble.zip"

destination = POPULATION_DIR / "pop_age_ensemble.zip"

print(f"\n📥 Downloading complete population dataset...")
print("This file contains population broken down by age groups for all French communes")

try:
    response = requests.get(url, timeout=60)
    response.raise_for_status()
    
    with open(destination, 'wb') as f:
        f.write(response.content)
    
    print(f"✅ Downloaded: {destination.name}")
    print(f"   Size: {destination.stat().st_size / 1_000_000:.2f} MB")
    
    # Extract
    import zipfile
    print("\n📦 Extracting...")
    with zipfile.ZipFile(destination, 'r') as zip_ref:
        zip_ref.extractall(POPULATION_DIR)
    
    print("✅ Extracted!")
    
    # Find the CSV
    csv_files = list(POPULATION_DIR.glob("*.csv"))
    print("\n📁 CSV files now available:")
    for f in csv_files:
        print(f"  - {f.name}")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nManual download: https://www.insee.fr/fr/statistiques/7739582")
    print("Download the 'ensemble.zip' file")

DOWNLOADING POPULATION BY AGE DATA

📥 Downloading complete population dataset...
This file contains population broken down by age groups for all French communes
✅ Downloaded: pop_age_ensemble.zip
   Size: 1.13 MB

📦 Extracting...
✅ Extracted!

📁 CSV files now available:
  - metadonnees.csv
  - donnees_communes.csv
  - donnees_collectivites.csv
  - donnees_arrondissements.csv
  - donnees_cantons.csv
  - donnees_communes_deleguees.csv
  - donnees_departements.csv
  - donnees_fractions_cantonales.csv
  - donnees_regions.csv


In [3]:
"""
Cell 7: Download POP1A - Population by sex and age groups
This has the elderly population data we need!
"""

import requests
import zipfile

print("=" * 60)
print("DOWNLOADING POP1A - POPULATION BY AGE GROUPS")
print("=" * 60)

# POP1A has 8 age groups including 65-79 and 80+
url = "https://www.insee.fr/fr/statistiques/fichier/7631680/TD_POP1A_2020_csv.zip"

destination = POPULATION_DIR / "TD_POP1A_2020.zip"

print(f"\n📥 Downloading POP1A dataset...")
print("Contains: Population by sex and age (8 classes)")
print("Age groups include: 65-79 years and 80+ years")

try:
    response = requests.get(url, timeout=90)
    response.raise_for_status()
    
    with open(destination, 'wb') as f:
        f.write(response.content)
    
    print(f"✅ Downloaded: {destination.name}")
    print(f"   Size: {destination.stat().st_size / 1_000_000:.2f} MB")
    
    # Extract
    print("\n📦 Extracting...")
    with zipfile.ZipFile(destination, 'r') as zip_ref:
        zip_ref.extractall(POPULATION_DIR / "POP1A")
    
    print("✅ Extracted to POP1A folder!")
    
    # List files
    pop1a_dir = POPULATION_DIR / "POP1A"
    print(f"\n📁 Files in POP1A:")
    for f in sorted(pop1a_dir.iterdir()):
        if f.is_file():
            size = f.stat().st_size / 1_000_000
            print(f"  - {f.name} ({size:.2f} MB)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nTry manual download from:")
    print("https://www.insee.fr/fr/statistiques/7631680?sommaire=7632456")
    print("Look for 'POP1A' CSV file (5 Mo)")

DOWNLOADING POP1A - POPULATION BY AGE GROUPS

📥 Downloading POP1A dataset...
Contains: Population by sex and age (8 classes)
Age groups include: 65-79 years and 80+ years
✅ Downloaded: TD_POP1A_2020.zip
   Size: 6.17 MB

📦 Extracting...
✅ Extracted to POP1A folder!

📁 Files in POP1A:
  - TD_POP1A_2020.csv (29.09 MB)


In [4]:
"""
Cell 8: Load POP1A and find elderly population columns
"""

print("=" * 60)
print("ANALYZING POPULATION BY AGE DATA")
print("=" * 60)

# Load the POP1A data
pop_file = POPULATION_DIR / "POP1A" / "TD_POP1A_2020.csv"

print(f"\n📂 Loading: {pop_file.name}")

# Load with semicolon separator (INSEE standard)
pop_data = pd.read_csv(pop_file, sep=';', low_memory=False)

print(f"✅ Loaded successfully!")
print(f"\n📊 Dataset shape:")
print(f"   - Rows: {len(pop_data):,}")
print(f"   - Columns: {len(pop_data.columns)}")

# Show first few columns
print(f"\n📋 First 20 columns:")
for i, col in enumerate(pop_data.columns[:20]):
    print(f"   {i+1}. {col}")

print(f"\n👀 Sample data (first 3 rows, first 10 columns):")
print(pop_data.iloc[:3, :10])

# Look for key columns
print(f"\n🔍 Looking for important columns:")

# Commune code column
commune_cols = [col for col in pop_data.columns if 'COM' in col.upper() or 'CODGEO' in col.upper()]
print(f"Commune identifier columns: {commune_cols}")

# Population 65+ columns (looking for P20 prefix and age groups)
age_cols = [col for col in pop_data.columns if ('P20' in col and '65' in col) or ('P20' in col and '80' in col)]
print(f"\nPopulation 65+ related columns (showing first 10):")
for col in age_cols[:10]:
    print(f"   - {col}")

# Total population column
total_cols = [col for col in pop_data.columns if 'P20_POP' in col and 'SEXE' not in col]
print(f"\nTotal population columns (first 5): {total_cols[:5]}")

print(f"\n💡 Next: We'll filter for Paris communes and calculate % elderly")

ANALYZING POPULATION BY AGE DATA

📂 Loading: TD_POP1A_2020.csv
✅ Loaded successfully!

📊 Dataset shape:
   - Rows: 680,213
   - Columns: 6

📋 First 20 columns:
   1. NIVGEO
   2. CODGEO
   3. LIBGEO
   4. SEXE
   5. AGEPYR10
   6. NB

👀 Sample data (first 3 rows, first 10 columns):
  NIVGEO CODGEO                   LIBGEO  SEXE  AGEPYR10         NB
0    COM  01001  L'Abergement-Clémenciat     1         0  18.070033
1    COM  01001  L'Abergement-Clémenciat     2         0  13.414280
2    COM  01001  L'Abergement-Clémenciat     1         3  14.402520

🔍 Looking for important columns:
Commune identifier columns: ['CODGEO']

Population 65+ related columns (showing first 10):

Total population columns (first 5): []

💡 Next: We'll filter for Paris communes and calculate % elderly


In [5]:
"""
Cell 9: Calculate elderly population (65+) by commune
"""

print("=" * 60)
print("CALCULATING ELDERLY POPULATION BY COMMUNE")
print("=" * 60)

# Filter for elderly (ages 65-79 and 80+)
# Values are integers, not strings!
elderly_data = pop_data[pop_data['AGEPYR10'].isin([65, 80])].copy()

print(f"\n📊 Elderly population records: {len(elderly_data):,}")

# Group by commune code (CODGEO) and sum population (NB column)
elderly_by_commune = elderly_data.groupby('CODGEO')['NB'].sum().reset_index()
elderly_by_commune.columns = ['CODGEO', 'pop_65plus']

print(f"✅ Calculated elderly population for {len(elderly_by_commune):,} communes")

# Also get total population by commune
total_by_commune = pop_data.groupby('CODGEO')['NB'].sum().reset_index()
total_by_commune.columns = ['CODGEO', 'pop_total']

# Merge
commune_pop = total_by_commune.merge(elderly_by_commune, on='CODGEO', how='left')
commune_pop['pop_65plus'] = commune_pop['pop_65plus'].fillna(0)

# Calculate percentage
commune_pop['pct_65plus'] = (commune_pop['pop_65plus'] / commune_pop['pop_total'] * 100).round(2)

print(f"\n📈 Summary statistics:")
print(commune_pop[['pop_total', 'pop_65plus', 'pct_65plus']].describe())

print(f"\n👀 Sample of data:")
print(commune_pop.head(10))

# Filter for Paris communes (75xxx) - exclude 75056 (overall Paris)
# Only keep arrondissements: 75101 to 75120
paris_communes = commune_pop[
    (commune_pop['CODGEO'].str.startswith('75')) & 
    (commune_pop['CODGEO'] != '75056')
].copy()

print(f"\n🗼 Paris arrondissements found: {len(paris_communes)}")
print("\nParis arrondissements:")
print(paris_communes.sort_values('CODGEO'))

# Show Paris statistics
if len(paris_communes) > 0:
    print(f"\n📊 Paris area statistics:")
    print(f"   - Total population: {paris_communes['pop_total'].sum():,.0f}")
    print(f"   - Population 65+: {paris_communes['pop_65plus'].sum():,.0f}")
    print(f"   - Average % 65+: {paris_communes['pct_65plus'].mean():.2f}%")
    print(f"   - Range: {paris_communes['pct_65plus'].min():.1f}% - {paris_communes['pct_65plus'].max():.1f}%")

print("\n✅ Data ready to join with heat zones!")

CALCULATING ELDERLY POPULATION BY COMMUNE

📊 Elderly population records: 138,437
✅ Calculated elderly population for 34,966 communes

📈 Summary statistics:
          pop_total     pop_65plus    pct_65plus
count  3.497300e+04   34973.000000  34967.000000
mean   2.021577e+03     405.321274     22.935656
std    1.537527e+04    2681.466482      7.713361
min    0.000000e+00       0.000000      0.000000
25%    1.970000e+02      43.733274     17.530000
50%    4.580000e+02      95.790706     21.750000
75%    1.165000e+03     248.303415     27.230000
max    2.145906e+06  367946.685877    100.000000

👀 Sample of data:
  CODGEO  pop_total   pop_65plus  pct_65plus
0  01001      806.0   158.068472       19.61
1  01002      262.0    44.011857       16.80
2  01004    14288.0  2584.528697       18.09
3  01005     1782.0   288.000000       16.16
4  01006      113.0    31.719298       28.07
5  01007     2827.0   465.770811       16.48
6  01008      768.0   105.327194       13.71
7  01009      324.0    9

In [12]:
"""
Cell 15: Calculate vulnerability scores from elderly population %
"""

print("=" * 60)
print("CALCULATING VULNERABILITY SCORES")
print("=" * 60)

# Use the Paris communes data we have
print(f"\n📊 Paris arrondissements data:")
print(paris_communes[['CODGEO', 'pop_total', 'pop_65plus', 'pct_65plus']].sort_values('CODGEO'))

# Calculate vulnerability score (0-10) based on % elderly
def calculate_vulnerability_score(pct_65plus):
    """
    Convert % population 65+ to vulnerability score (0-10)
    
    Thresholds based on French demographics:
    - <10%: Very low (score 1-2)
    - 10-15%: Low (score 3-4)  
    - 15-20%: Moderate (score 5-6)
    - 20-25%: High (score 7-8)
    - >25%: Very high (score 9-10)
    """
    if pct_65plus < 10:
        return 2
    elif pct_65plus < 15:
        return 4
    elif pct_65plus < 20:
        return 6
    elif pct_65plus < 25:
        return 8
    else:
        return 10

# Apply vulnerability scoring
paris_communes['vulnerability_score'] = paris_communes['pct_65plus'].apply(calculate_vulnerability_score)

print(f"\n🎯 Vulnerability scores calculated!")
print("\nArrondissements with vulnerability scores:")
print(paris_communes[['CODGEO', 'pct_65plus', 'vulnerability_score']].sort_values('vulnerability_score', ascending=False))

# Save this data for the Streamlit app
vulnerability_file = PROCESSED_DATA_DIR / "paris_vulnerability.csv"
paris_communes.to_csv(vulnerability_file, index=False)

print(f"\n💾 Saved vulnerability data to: {vulnerability_file.name}")

print(f"\n📈 Vulnerability summary:")
print(f"   - Average vulnerability score: {paris_communes['vulnerability_score'].mean():.2f}/10")
print(f"   - Highest risk arrondissement: {paris_communes.loc[paris_communes['vulnerability_score'].idxmax(), 'CODGEO']}")
print(f"   - Lowest risk arrondissement: {paris_communes.loc[paris_communes['vulnerability_score'].idxmin(), 'CODGEO']}")

print("\n✅ Ready to integrate into Streamlit app!")
print("\nNext steps:")
print("1. Update Streamlit app to load vulnerability data")
print("2. Display vulnerability alongside heat exposure")
print("3. (Future) Do proper spatial join for composite risk at zone level")


CALCULATING VULNERABILITY SCORES

📊 Paris arrondissements data:
      CODGEO  pop_total    pop_65plus  pct_65plus
29278  75101    16030.0   2868.896146       17.90
29279  75102    21130.0   2304.755530       10.91
29280  75103    33402.0   5169.856817       15.48
29281  75104    29064.0   4977.933860       17.13
29282  75105    57380.0  11517.268571       20.07
29283  75106    39625.0   9467.054282       23.89
29284  75107    48520.0  11484.582610       23.67
29285  75108    35631.0   6155.850869       17.28
29286  75109    60168.0   8070.879302       13.41
29287  75110    83459.0  10745.839716       12.88
29288  75111   144292.0  22468.843780       15.57
29289  75112   140311.0  24972.672213       17.80
29290  75113   177833.0  32195.436834       18.10
29291  75114   133967.0  25052.344553       18.70
29292  75115   229472.0  41993.909060       18.30
29293  75116   162820.0  36602.616766       22.48
29294  75117   166336.0  26397.796680       15.87
29295  75118   191135.0  26357.75018